# Pahlavi Corpus Builder

Links:
- [extracting MS Word data](https://towardsdatascience.com/how-to-extract-data-from-ms-word-documents-using-python-ed3fbb48c122)
- [navigating MS Word XML data](https://virantha.com/2013/08/16/reading-and-writing-microsoft-word-docx-files-with-python/)

In [1]:
import os, zipfile, re, glob
from bs4 import BeautifulSoup

In [2]:
#set home directory path
hdir = os.path.expanduser('~')

#pahlavi corpus directory
pah_path = hdir + "/Box/Notes/Digital_Humanities/Corpora/pahlavi_corpus/"

Glob the corpus

In [3]:
pah_files = glob.glob(pah_path + r'/*.docx')

pah_xml_corpus = {}
for longname in pah_files:
    document = zipfile.ZipFile(longname)
    txt = zipfile.ZipFile.read(document, 'word/document.xml', pwd=None)
    start = os.path.basename(longname)
    short = os.path.splitext(start)
    pah_xml_corpus[short[0]] = txt

Assemble simple corpus divided by MS Word paragraph breaks

In [4]:
pahlavi_corpus = {}
for work in pah_xml_corpus:
    tree = BeautifulSoup(pah_xml_corpus[work])
    paras = tree.find_all("w:p")
    document = {}
    for i in range(len(paras)):
        if len(paras[i].get_text()) > 0:
            document[i] = paras[i].get_text()
    pahlavi_corpus[work] = document

In [14]:
#pah_xml_corpus['Dēnkard 1']

pahlavi_corpus["Dēnkard 5"][100]

'5.9.6nē padīrēnd nē kerdan kōšišn ī abāg dēwān wattarān ud rāyēnīdan paywand warzīdan ī gēhān tāšišnīh pad abārīg frārōnīh čiyōn hangerdīg anayārīh nām '

Extracting the Line Numbers

In [41]:
pahlavi_corpus_lines = {}
for work in pahlavi_corpus:
    segment = {}
    for para in pahlavi_corpus[work]:
        num_pattern = re.compile(r"^[0-9]{0,3}\.[0-9]{0,3}\.?[0-9]{0,3}")
        num_match = re.match(num_pattern, pahlavi_corpus[work][para])
        if num_match:
            num = num_match.group(0)
        else:
            num = "--"
        line_pattern = re.compile(r"[^(0-9|\.)]*")
        line_match = re.match(line_pattern, pahlavi_corpus[work][para])
        if line_match:
            line = line_match.group(0)
        else:
            line = "-----"
        segment[para] = [num, line]
    pahlavi_corpus_lines[work] = segment
    

In [27]:
test_string = "5.23.5ud čē rāy pēš nē āmad"
num_pattern = re.compile(r"^[0-9]{0,3}\.[0-9]{0,3}\.?[0-9]{0,3}")
result = re.match(num_pattern, test_string)
print(result.group(0))

5.23.5


In [43]:
pahlavi_corpus_lines["Dēnkard 5"][103]



['5.9.9', '']

In [69]:
num_pattern = re.compile(r"hanger", re.UNICODE)
num = re.match(num_pattern, pahlavi_corpus["Dēnkard 5"][100]).group(0)
num

AttributeError: 'NoneType' object has no attribute 'group'

Pickling for export